In [53]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from newsapi import NewsApiClient
from datetime import datetime, timedelta
import numpy as np
from numpy.random import seed
seed(1)
import requests
from tensorflow import random
random.set_seed(2)
import hvplot.pandas
import matplotlib.pyplot as plt
%matplotlib inline

In [54]:
load_dotenv()

True

In [55]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")
#print(alpaca_api_key)

In [56]:
#15Min interval
timeframe = "1D"
start_date = pd.Timestamp("2018-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-12-31", tz="America/New_York").isoformat()
#today = pd.Timestamp("2021-01-02", tz="America/New_York").isoformat()
ticker = ["PYPL", "SQ", "MELI"]
fintech_ticker = alpaca.get_barset(
    ticker,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df
fintech_ticker.dropna(inplace=True)
fintech_ticker.head(5)
#apple_ticker.dropna(inplace=True)

MELI                                        PYPL  \
                             open     high      low   close    volume    open   
time                                                                            
2018-01-02 00:00:00-05:00  317.49  322.635  316.990  322.58  248381.0  74.235   
2018-01-03 00:00:00-05:00  324.67  329.970  322.500  329.24  339110.0  74.300   
2018-01-04 00:00:00-05:00  329.89  332.940  322.550  325.88  299970.0  77.330   
2018-01-05 00:00:00-05:00  329.20  336.485  328.595  330.64  389591.0  77.110   
2018-01-08 00:00:00-05:00  328.12  341.470  327.660  340.50  272992.0  78.700   

                                                                SQ         \
                              high      low  close   volume   open   high   
time                                                                        
2018-01-02 00:00:00-05:00  74.5000  73.4300  73.83  6009642  34.70  36.24   
2018-01-03 00:00:00-05:00  76.8499  74.0500  76.77  8012610  36.43  37.36   
2018-01-04 00:00:00-05:00  77.9287  76.2599  76.72  7246995  38.30  38.66   
2018-01-05 00:00:00-05:00  78.9300  77.0100  78.64  9758882  38.39  41.15   
2018-01-08 00:00:00-05:00  79.1800  77.7730  79.04  5761316  41.00  41.91   

                                                     
                             low    close    volume  
time                                                 
2018-01-02 00:00:00-05:00  34.68  36.1786  10359616  
2018-01-03 00:00:00-05:00  36.05  37.3100  10537507  
2018-01-04 00:00:00-05:00  37.40  38.1000  18843623  
2018-01-05 00:00:00-05:00  38.19  41.1400  28787543  
2018-01-08 00:00:00-05:00  39.81  40.7700  19693914

In [57]:
fintech_df = pd.DataFrame()
fintech_df["PayPal"] = fintech_ticker["PYPL"]["close"]
fintech_df["Square"] = fintech_ticker["SQ"]["close"]
#fintech_df["Affirm"] = fintech_ticker["AFRM"]["close"]
fintech_df["Mercadolibre"] = fintech_ticker["MELI"]["close"]
#fintech_df["Johnson&Johnson"] = nasdaq_ticker["JNJ"]["close"]
fintech_df.index = fintech_df.index.date 
fintech_returns = fintech_df.pct_change().dropna()
#fintech_df.dropna(inplace=True)
fintech_returns.head(10)

,PayPal,Square,Mercadolibre
2018-01-03,0.039821,0.031273,0.020646
2018-01-04,-0.000651,0.021174,-0.010205
2018-01-05,0.025026,0.079790,0.014607
2018-01-08,0.005086,-0.008994,0.029821
2018-01-09,0.002024,0.002610,-0.009163
2018-01-10,0.002020,-0.000890,0.006550
2018-01-11,0.004788,0.035504,0.007126
2018-01-12,0.009155,-0.024592,0.001345
2018-01-16,-0.000621,-0.046788,-0.007738
2018-01-17,0.025616,0.019074,0.001854


In [58]:
fintech_std = fintech_returns.rolling(window=21).std()
fintech_std.hvplot.line(label = "Standard Deviation Between 2018 - 2021", xlabel = "Date", ylabel = "Price")

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [59]:
csv_path = Path("./stock_data/first_half.csv/")
first_half_data = pd.read_csv(csv_path, index_col = "Date")
first_half_data.sort_index(ascending = True, inplace = True)
first_half_data.tail()

,PayPal
Date,
2017-12-22,73.89
2017-12-26,74.28
2017-12-27,74.58
2017-12-28,74.17
2017-12-29,73.63


In [60]:
csv_path = Path("./stock_data/second_half.csv/")
second_half_data = pd.read_csv(csv_path, index_col = "Date")
second_half_data.sort_index(ascending = True, inplace = True)
second_half_data.head()

,PayPal
Date,
2018-01-02,73.83
2018-01-03,76.77
2018-01-04,76.72
2018-01-05,78.64
2018-01-08,79.04


In [61]:
fintech = first_half_data.append(second_half_data)

In [62]:
fintech

,PayPal
Date,
2015-07-06,36.71
2015-07-07,36.55
2015-07-08,34.68
2015-07-09,34.49
2015-07-10,34.89
...,...
2021-08-17,271.10
2021-08-18,268.84
2021-08-19,270.53


In [63]:
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window):
        features = df.iloc[i : (i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [64]:
window_size = 10
feature_column = 0
target_column = 0
X, y = window_data(fintech, window_size, feature_column, target_column)
print (f"X sample values:\n{X[:5]} \n")
print (f"y sample values:\n{y[:5]}")

X sample values:
[[36.71  36.55  34.68  34.49  34.89  36.82  36.9   37.5   38.675 38.62 ]
 [36.55  34.68  34.49  34.89  36.82  36.9   37.5   38.675 38.62  40.479]
 [34.68  34.49  34.89  36.82  36.9   37.5   38.675 38.62  40.479 39.35 ]
 [34.49  34.89  36.82  36.9   37.5   38.675 38.62  40.479 39.35  38.38 ]
 [34.89  36.82  36.9   37.5   38.675 38.62  40.479 39.35  38.38  37.01 ]] 

y sample values:
[[40.479]
 [39.35 ]
 [38.38 ]
 [37.01 ]
 [37.025]]


In [65]:
split = int(0.7 * len(X))
X_train = X[: split]
X_test = X[split :]
y_train = y[: split]
y_test = y[split :]

In [66]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
scaler.fit(y)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [67]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print (f"X_train sample values:\n{X_train[:5]} \n")
print (f"X_test sample values:\n{X_test[:5]}")

X_train sample values:
[[[0.0218831 ]
  [0.02130723]
  [0.01457673]
  [0.01389289]
  [0.01533257]
  [0.02227901]
  [0.02256695]
  [0.02472646]
  [0.02895551]
  [0.02875756]]

 [[0.02130723]
  [0.01457673]
  [0.01389289]
  [0.01533257]
  [0.02227901]
  [0.02256695]
  [0.02472646]
  [0.02895551]
  [0.02875756]
  [0.03544846]]

 [[0.01457673]
  [0.01389289]
  [0.01533257]
  [0.02227901]
  [0.02256695]
  [0.02472646]
  [0.02895551]
  [0.02875756]
  [0.03544846]
  [0.03138497]]

 [[0.01389289]
  [0.01533257]
  [0.02227901]
  [0.02256695]
  [0.02472646]
  [0.02895551]
  [0.02875756]
  [0.03544846]
  [0.03138497]
  [0.02789375]]

 [[0.01533257]
  [0.02227901]
  [0.02256695]
  [0.02472646]
  [0.02895551]
  [0.02875756]
  [0.03544846]
  [0.03138497]
  [0.02789375]
  [0.02296286]]] 

X_test sample values:
[[[0.25169162]
  [0.25525482]
  [0.25604665]
  [0.2624892 ]
  [0.26094155]
  [0.26284912]
  [0.25410308]
  [0.2548949 ]
  [0.2401742 ]
  [0.23747481]]

 [[0.25525482]
  [0.25604665]
  [0.262489

In [68]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [69]:
model = Sequential()

#Adding number_units to the architecture enables the model to far more accurate that with a single digit number
number_units = 30
dropout_fraction = 0.2


# Layer 1
model.add(LSTM(units=number_units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(dropout_fraction))
# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))
# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))
# Output layer
model.add(Dense(1))

In [70]:
model.compile(optimizer="adam", loss="mean_squared_error")

In [71]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10, 30)            3840      
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 30)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 10, 30)            7320      
_________________________________________________________________
dropout_4 (Dropout)          (None, 10, 30)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 30)                7320      
_________________________________________________________________
dropout_5 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

In [72]:
model.fit(X_train, y_train, epochs=20, shuffle=False, batch_size=1, verbose=1)

Epoch 1/20
1075/1075 [==============================] - 9s 5ms/step - loss: 2.0951e-04
Epoch 2/20
1075/1075 [==============================] - 5s 5ms/step - loss: 4.8240e-04
Epoch 3/20
1075/1075 [==============================] - 5s 5ms/step - loss: 5.7247e-04
Epoch 4/20
1075/1075 [==============================] - 5s 5ms/step - loss: 5.7651e-04
Epoch 5/20
1075/1075 [==============================] - 5s 5ms/step - loss: 6.4241e-04
Epoch 6/20
1075/1075 [==============================] - 5s 5ms/step - loss: 6.6373e-04
Epoch 7/20
1075/1075 [==============================] - 5s 5ms/step - loss: 6.2491e-04
Epoch 8/20
1075/1075 [==============================] - 5s 5ms/step - loss: 8.0129e-04
Epoch 9/20
1075/1075 [==============================] - 5s 5ms/step - loss: 7.6428e-04
Epoch 10/20
1075/1075 [==============================] - 6s 5ms/step - loss: 7.8726e-04
Epoch 11/20
1075/1075 [==============================] - 7s 6ms/step - loss: 7.3132e-04
Epoch 12/20
1075/1075 [==================

In [73]:
model.evaluate(X_test, y_test)

15/15 [==============================] - 1s 3ms/step - loss: 0.1600 


0.1600121408700943

In [74]:
predicted = model.predict(X_

In [75]:
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [76]:
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
    }, index = fintech.index[-len(real_prices): ])
stocks.head(10)

,Real,Predicted
Date,,
2019-10-24,104.91,101.707687
2019-10-25,107.20,101.706841
2019-10-28,106.62,101.709900
2019-10-29,105.00,101.714302
2019-10-30,106.52,101.717949
2019-10-31,104.05,101.720802
2019-11-01,105.00,101.722321
2019-11-04,102.79,101.722687
2019-11-05,101.00,101.721931


In [77]:
stocks.hvplot.line(label = "LSTM Model", width = 1000)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)